In [10]:
import cv2
from mediapipe.python.solutions import hands as mp_hands
from mediapipe.python.solutions import drawing_utils as mp_draw
import pyautogui
import time

# Initialize MediaPipe Hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)

# Function to determine which fingers are up
def get_finger_states(hand_landmarks):
    fingers = []

    # Thumb: compare tip (4) to joint (3) - for right hand only, flip if needed
    if hand_landmarks.landmark[4].x < hand_landmarks.landmark[3].x:
        fingers.append(1)  # Thumb up
    else:
        fingers.append(0)  # Thumb down

    # Fingers: index(8), middle(12), ring(16), pinky(20)
    tip_ids = [8, 12, 16, 20]
    for tip in tip_ids:
        if hand_landmarks.landmark[tip].y < hand_landmarks.landmark[tip - 2].y:
            fingers.append(1)
        else:
            fingers.append(0)

    return fingers  # [thumb, index, middle, ring, pinky]

# Map finger states to gesture and trigger keyboard commands for Temple Run
def interpret_gesture(fingers):
    # Only three gestures, simplified:

    # Index finger only → Jump (space key)
    if fingers == [0, 1, 0, 0, 0]:
        pyautogui.press('space')
        return "Index ➜ Jump"

    # Thumb only → Slide (down arrow)
    elif fingers == [1, 0, 0, 0, 0]:
        pyautogui.press('down')
        return "Thumb ➜ Slide"

    # Index + Thumb → Move Left (left arrow)
    elif fingers == [1, 1, 0, 0, 0]:
        pyautogui.press('left')
        return "Index + Thumb ➜ Left"

    else:
        return "Show a gesture..."

# Start webcam
cap = cv2.VideoCapture(0)
time.sleep(2)
print("🖐️ Show thumb & index gestures to control Temple Run. Press ESC to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)  # Mirror image
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(rgb)

    gesture_text = "No hand detected"

    if result.multi_hand_landmarks:
        for handLms in result.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, handLms, mp_hands.HAND_CONNECTIONS)

            finger_states = get_finger_states(handLms)
            gesture_text = interpret_gesture(finger_states)

    # Draw transparent rectangle behind text
    overlay = frame.copy()
    cv2.rectangle(overlay, (5, 5), (400, 50), (0, 0, 0), -1)
    alpha = 0.6  # Transparency factor
    frame = cv2.addWeighted(overlay, alpha, frame, 1 - alpha, 0)

    # Display gesture text
    cv2.putText(frame, gesture_text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX,
                1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("Temple Run Hand Gesture Control", frame)

    # Exit on ESC key
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


🖐️ Show thumb & index gestures to control Temple Run. Press ESC to exit.
